In [1]:
import requests
from urllib.request import urlopen
import pandas as pd
import numpy as np
import MySQLdb as sql

## 簡単なデータをMySQLに格納

In [2]:
#DBを作成して、適当なデータをMySQLに格納
password = open("password.txt").read()
connect = sql.connect(db="test_db", user="root", passwd=password, host="localhost", charset="utf8mb4")
c = connect.cursor()   #カーソルを取得

In [26]:
#新しいテーブルを作成
c.execute("DROP TABLE IF EXISTS cities")   #citiesテーブルがある場合、テーブルを削除
c.execute("CREATE TABLE cities(\
            id int PRIMARY KEY AUTO_INCREMENT,\
            city varchar(32),\
            popuration int)")

0

In [27]:
#データを格納
sql = "INSERT INTO cities VALUES (%s, %s, %s)"
c.execute(sql, (1, "上海", 24150000))   #パラメータで置き換える場所を指定

#複数のデータを登録
datas = [(2, "カラチ", 23500000),
         (3, "北京", 21516000),
         (4, "天津", 14722100),
         (5, "イスタンブール", 14160467)]
c.executemany(sql, datas)
connect.commit()   #変更をコミットする

In [45]:
#データを確認
c.execute("SELECT * FROM cities;")
c.fetchall()

((1, '上海', 24150000),
 (2, 'カラチ', 23500000),
 (3, '北京', 21516000),
 (4, '天津', 14722100),
 (5, 'イスタンブール', 14160467))

## pandasデータフレームからMySQLに格納

In [174]:
#適当なデータを作成
n = 1000
k = 5
df = pd.concat([pd.DataFrame(np.arange(n)*1, columns=["id"]), 
                pd.DataFrame(np.random.normal(0, 1, n*k).reshape(n, k), columns=["a", "b", "c", "d", "e"])], axis=1)
id = np.array(df.iloc[:, 0]) + 1
dt = df.iloc[:, 1:].values
df[0:5]

,id,a,b,c,d,e
0,0,-0.022279,0.633146,0.137561,-0.520150,-1.134804
1,1,0.429021,-0.590135,0.394109,0.063192,-1.007226
2,2,-0.076277,-0.763587,-1.949166,-1.090949,-0.671110
3,3,-0.205958,0.309136,-0.046655,-0.811027,0.128210
4,4,-0.542950,1.611744,0.753801,-0.232803,-0.830537


In [175]:
#SQLに格納するためのリストを作成
datas = [i for i in range(n)]
for i in range(n):
    datas[i] = (id[i], ) + tuple(dt[i, ])

In [177]:
#新しいテーブルを作成
c.execute("DROP TABLE IF EXISTS test_table")   #citiesテーブルがある場合、テーブルを削除
c.execute("CREATE TABLE test_table(\
            id int(20) PRIMARY KEY AUTO_INCREMENT,\
            a float,\
            b float,\
            c float,\
            d float,\
            e float)")

0

In [178]:
#データを格納
sql = "INSERT INTO test_table VALUES (%s, %s, %s, %s, %s, %s)"
c.executemany(sql, datas)
connect.commit()   #変更をコミットする

In [179]:
#データを確認
c.execute("SELECT * FROM test_table;")
c.fetchall()

((1, -0.0222794, 0.633146, 0.137561, -0.52015, -1.1348),
 (2, 0.429021, -0.590136, 0.394109, 0.0631923, -1.00723),
 (3, -0.0762769, -0.763587, -1.94917, -1.09095, -0.67111),
 (4, -0.205958, 0.309136, -0.0466554, -0.811027, 0.12821),
 (5, -0.54295, 1.61174, 0.753801, -0.232803, -0.830537),
 (6, 2.07228, 0.543232, 0.165029, 0.337274, -1.32396),
 (7, -0.299553, -1.09821, 0.417639, -1.11467, -1.08075),
 (8, -1.2313, -2.4435, 0.324151, -0.447403, -0.968241),
 (9, 0.326443, 1.42914, 0.914928, -0.952003, -0.359936),
 (10, 0.708026, 0.450599, 0.648375, -0.222644, 0.0499491),
 (11, -0.445129, 0.325146, 0.384234, -0.454198, 0.270682),
 (12, -0.251226, 1.34869, -0.181332, 0.14511, -0.0860666),
 (13, -1.22992, 0.338582, 1.31256, 0.6269, 0.179169),
 (14, 1.46789, 2.39263, -0.918034, 0.756807, 1.39761),
 (15, -0.811759, 0.575073, -0.0329293, -0.807741, 0.561663),
 (16, 0.176103, -0.429416, -0.653836, 1.73006, 1.06085),
 (17, 0.682894, 0.144118, -0.183047, 0.48564, 0.40018),
 (18, -0.594323, -0.53169